# Imports

In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from controller import Controller

In [3]:
c = Controller('i01')

variables = [
    'project_id',
    'name', 
    'desc',
    'goal',
    'keywords',
    'disable_communication',
    'country',
    'currency',
    'deadline',
    'state_changed_at',
    'created_at',
    'launched_at',
    'backers_count',
]

# Load data

In [4]:
datecols = ['deadline', 'state_changed_at', 'created_at', 'launched_at']

df_train_val_test_orig = pd.read_csv(fr'{c.get_path_data_original()}/train.csv')
df_train_val_test_orig[datecols] = df_train_val_test_orig[datecols].apply(pd.to_datetime)

df_predict = pd.read_csv(fr'{c.get_path_data_original()}/test.csv')
df_predict[datecols] = df_predict[datecols].apply(pd.to_datetime)

print(f'{df_train_val_test_orig.shape=}')
print(f'{df_predict.shape=}')

df_train_val_test_orig.shape=(108129, 14)
df_predict.shape=(63465, 12)


In [5]:
df_train_val_test_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108129 entries, 0 to 108128
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   project_id             108129 non-null  object        
 1   name                   108126 non-null  object        
 2   desc                   108120 non-null  object        
 3   goal                   108129 non-null  float64       
 4   keywords               108129 non-null  object        
 5   disable_communication  108129 non-null  bool          
 6   country                108129 non-null  object        
 7   currency               108129 non-null  object        
 8   deadline               108129 non-null  datetime64[ns]
 9   state_changed_at       108129 non-null  datetime64[ns]
 10  created_at             108129 non-null  datetime64[ns]
 11  launched_at            108129 non-null  datetime64[ns]
 12  backers_count          108129 non-null  int6

In [6]:
# `final_status` is the target variable.
# Note that `backers_count` is missing. This means we shouldn't train the model on it.
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63465 entries, 0 to 63464
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   project_id             63465 non-null  object        
 1   name                   63465 non-null  object        
 2   desc                   63461 non-null  object        
 3   goal                   63465 non-null  float64       
 4   keywords               63465 non-null  object        
 5   disable_communication  63465 non-null  bool          
 6   country                63465 non-null  object        
 7   currency               63465 non-null  object        
 8   deadline               63465 non-null  datetime64[ns]
 9   state_changed_at       63465 non-null  datetime64[ns]
 10  created_at             63465 non-null  datetime64[ns]
 11  launched_at            63465 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](4), float64(1), object(6)
memory 

# Check for mixed types

In [7]:
can_be_numeric = []

for col in df_train_val_test_orig.select_dtypes(exclude='number'):
    try:
        pd.to_numeric(df_train_val_test_orig[col])
    except ValueError:
        pass
    else:
        can_be_numeric.append(col)

# We are left with the date columns which is ok.
can_be_numeric

['disable_communication',
 'deadline',
 'state_changed_at',
 'created_at',
 'launched_at']

# Train-val-test split

In [8]:
df_train, df_val_test = train_test_split(df_train_val_test_orig, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_val_test, test_size=0.5, random_state=42)

print(f'{df_train.shape=}')
print(f'{df_val.shape=}')
print(f'{df_test.shape=}')

df_train['TRAIN_VAL_TEST_SPLIT'] = 'Train'
df_val['TRAIN_VAL_TEST_SPLIT'] = 'Val'
df_test['TRAIN_VAL_TEST_SPLIT'] = 'Test'

df_train_val_test = pd.concat([df_train, df_val, df_test], ignore_index=True)

pd.concat([
    df_train_val_test['TRAIN_VAL_TEST_SPLIT'].value_counts(dropna=False),
    df_train_val_test['TRAIN_VAL_TEST_SPLIT'].value_counts(dropna=False, normalize=True),
], axis=1, keys=['Total #', 'Total %'])

df_train.shape=(86503, 14)
df_val.shape=(10813, 14)
df_test.shape=(10813, 14)


,Total #,Total %
Train,86503,0.799998
Val,10813,0.100001
Test,10813,0.100001


# Check for duplicates

In [9]:
# `False` means no duplicates
any(df_train_val_test.duplicated())

False

# Capitalize column names

In [10]:
df_train_val_test.columns = df_train_val_test.columns.map(lambda col: col.upper())
df_train_val_test.columns

Index(['PROJECT_ID', 'NAME', 'DESC', 'GOAL', 'KEYWORDS',
       'DISABLE_COMMUNICATION', 'COUNTRY', 'CURRENCY', 'DEADLINE',
       'STATE_CHANGED_AT', 'CREATED_AT', 'LAUNCHED_AT', 'BACKERS_COUNT',
       'FINAL_STATUS', 'TRAIN_VAL_TEST_SPLIT'],
      dtype='object')

# Frequency of missing values

In [11]:
pd.concat([
    df_train_val_test.isna().sum().sort_values(ascending=False),
    df_train_val_test.isna().mean(),
], axis=1, keys=['# Missing', '% Missing'])

,# Missing,% Missing
DESC,9,0.000083
NAME,3,0.000028
PROJECT_ID,0,0.000000
GOAL,0,0.000000
KEYWORDS,0,0.000000
DISABLE_COMMUNICATION,0,0.000000
COUNTRY,0,0.000000
CURRENCY,0,0.000000
DEADLINE,0,0.000000
STATE_CHANGED_AT,0,0.000000


# Plot flag frequencies

In [12]:
column = 'FINAL_STATUS'
pd.concat([
    df_train_val_test[column].value_counts(dropna=False),
    df_train_val_test[column].value_counts(dropna=False, normalize=True),
], axis=1, keys=['Total #', 'Total %'])

,Total #,Total %
0,73568,0.680373
1,34561,0.319627


# Saving to files

In [13]:
filepath = fr'{c.get_path_data_prepared()}/01_df_train_val_test.pkl'
df_train_val_test.to_pickle(filepath)